In [1]:
# ============================== SETUP ==============================
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

In [2]:
# I/O
DATA_PATH = "Dataset .csv"
OUT_DIR   = "outputs_level3"
os.makedirs(OUT_DIR, exist_ok=True)

In [3]:
# Read
df = pd.read_csv(DATA_PATH)

# Quick sanity
print("Shape:", df.shape)
print("Columns:", list(df.columns))

Shape: (9551, 21)
Columns: ['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address', 'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines', 'Average Cost for two', 'Currency', 'Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu', 'Price range', 'Aggregate rating', 'Rating color', 'Rating text', 'Votes']


In [4]:
# ============================== COMMON CLEANING ==============================
# Standardize Yes/No flags for booking/delivery (some datasets use 'Yes'/'No')
def yn_to_int(x):
    x = str(x).strip().lower()
    return 1 if x == "yes" else 0

for col in ["Has Table booking", "Has Online delivery", "Is delivering now", "Switch to order menu"]:
    if col in df.columns:
        df[col] = df[col].apply(yn_to_int)

In [5]:
# Ensure numeric types where needed
for col in ["Aggregate rating", "Votes", "Price range", "Average Cost for two", "Latitude", "Longitude"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Drop obvious rating nulls (target)
df = df.dropna(subset=["Aggregate rating"]).copy()


In [6]:
# Fill remaining numeric/categorical nulls (simple strategy)
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = [c for c in df.columns if c not in num_cols]
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
for c in cat_cols:
    df[c] = df[c].fillna("Unknown")



In [8]:
# ============================== TASK 2: CUSTOMER PREFERENCE ANALYSIS ==============================
# --- Explode multi-label "Cuisines" into rows for per-cuisine stats
def split_cuisines(s):
    if pd.isna(s): return []
    return [t.strip() for t in str(s).split(",") if t.strip()]

exploded = df[["Cuisines", "Aggregate rating", "Votes"]].copy()
exploded["Cuisine"] = exploded["Cuisines"].apply(lambda s: split_cuisines(s))
exploded = exploded.explode("Cuisine")
exploded = exploded.dropna(subset=["Cuisine"])

In [9]:
# Overall stats per cuisine
cuisine_stats = (
    exploded.groupby("Cuisine")
    .agg(
        n_restaurants = ("Cuisine", "count"),
        avg_rating    = ("Aggregate rating", "mean"),
        total_votes   = ("Votes", "sum"),
        avg_votes     = ("Votes", "mean"),
    )
    .sort_values(["avg_rating", "n_restaurants"], ascending=[False, False])
)

In [10]:
# Popular cuisines by votes (top 20)
popular_by_votes = cuisine_stats.sort_values("total_votes", ascending=False).head(20)

# High-rating cuisines with support threshold (e.g., ≥ 30 restaurants)
MIN_SUPPORT = 30
high_rating_cuisines = cuisine_stats[cuisine_stats["n_restaurants"] >= MIN_SUPPORT] \
                          .sort_values("avg_rating", ascending=False).head(20)

In [11]:
# Save tables
cuisine_stats.to_csv(f"{OUT_DIR}/cuisine_stats_full.csv")
popular_by_votes.to_csv(f"{OUT_DIR}/popular_cuisines_by_votes.csv")
high_rating_cuisines.to_csv(f"{OUT_DIR}/high_rating_cuisines_min{MIN_SUPPORT}.csv")
print("Saved cuisine analyses CSVs.")


Saved cuisine analyses CSVs.


In [12]:
# Plots
plt.figure()
popular_by_votes["total_votes"].iloc[::-1].plot(kind="barh")
plt.title("Most Popular Cuisines by Total Votes (Top 20)")
plt.xlabel("Total Votes")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/popular_cuisines_votes.png", dpi=150); plt.close()

plt.figure()
high_rating_cuisines["avg_rating"].iloc[::-1].plot(kind="barh")
plt.title(f"Highest Avg Rating Cuisines (min {MIN_SUPPORT} restaurants)")
plt.xlabel("Average Rating")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/high_rating_cuisines.png", dpi=150); plt.close()

print("Saved cuisine analysis plots.")

Saved cuisine analysis plots.


In [13]:
# ============================== TASK 3: DATA VISUALIZATION ==============================
# 1) Distribution of ratings
plt.figure()
df["Aggregate rating"].plot(kind="hist", bins=20)
plt.title("Distribution of Aggregate Ratings")
plt.xlabel("Aggregate rating")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/rating_histogram.png", dpi=150); plt.close()


In [14]:
# 2) Average rating by city (Top 25 cities by count)
if "City" in df.columns:
    top_cities = df["City"].value_counts().head(25).index
    city_avg = df[df["City"].isin(top_cities)].groupby("City")["Aggregate rating"].mean().sort_values()
    plt.figure()
    city_avg.plot(kind="barh")
    plt.title("Average Rating by City (Top 25 by Count)")
    plt.xlabel("Average Rating")
    plt.tight_layout()
    plt.savefig(f"{OUT_DIR}/avg_rating_by_city.png", dpi=150); plt.close()


In [16]:
# 3) Average rating by price range
if "Price range" in df.columns:
    pr_avg = df.groupby("Price range")["Aggregate rating"].mean()
    plt.figure()
    pr_avg.plot(kind="bar")
    plt.title("Average Rating by Price Range")
    plt.xlabel("Price Range")
    plt.ylabel("Average Rating")
    plt.tight_layout()
    plt.savefig(f"{OUT_DIR}/avg_rating_by_price_range.png", dpi=150); plt.close()

In [17]:
# 4) Relationship examples
#    Votes vs Rating (scatter)
plt.figure()
plt.scatter(df["Votes"], df["Aggregate rating"], s=8, alpha=0.5)
plt.title("Votes vs Aggregate Rating")
plt.xlabel("Votes")
plt.ylabel("Aggregate rating")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/votes_vs_rating.png", dpi=150); plt.close()

In [18]:
#    Price vs Rating (box)
if "Price range" in df.columns:
    plt.figure()
    df.boxplot(column="Aggregate rating", by="Price range")
    plt.suptitle("")
    plt.title("Aggregate Rating by Price Range")
    plt.xlabel("Price range")
    plt.ylabel("Aggregate rating")
    plt.tight_layout()
    plt.savefig(f"{OUT_DIR}/rating_by_price_box.png", dpi=150); plt.close()

print("Saved visualizations.")



Saved visualizations.


<Figure size 640x480 with 0 Axes>

In [19]:
# ============================== TASK 1: PREDICTIVE MODELING ==============================
# Goal: Predict Aggregate rating from available features

# -------- Feature Engineering (lightweight, robust) --------
# Primary cuisine (first token)
def primary_cuisine(s):
    toks = split_cuisines(s)
    return toks[0] if toks else "Unknown"

df["Primary Cuisine"] = df["Cuisines"].apply(primary_cuisine)

# Name/Address lengths & cuisine count
df["Name_length"] = df["Restaurant Name"].astype(str).str.len()
df["Address_length"] = df["Address"].astype(str).str.len()
df["Cuisine_count"] = df["Cuisines"].apply(lambda s: len(split_cuisines(s)))


In [20]:
# Keep a compact but informative feature set
feature_cols_num = [
    c for c in [
        "Votes", "Price range", "Average Cost for two",
        "Latitude", "Longitude",
        "Has Table booking", "Has Online delivery",
        "Is delivering now", "Switch to order menu",
        "Name_length", "Address_length", "Cuisine_count"
    ] if c in df.columns
]

In [21]:
feature_cols_cat = [c for c in [
    # treat 'Country Code' as categorical (IDs), keep it numeric but one-hot
    "Country Code" if "Country Code" in df.columns else None,
    "City" if "City" in df.columns else None,
    "Primary Cuisine",
    "Currency" if "Currency" in df.columns else None,
    "Rating color" if "Rating color" in df.columns else None
] if c is not None]

X = df[feature_cols_num + feature_cols_cat].copy()
y = df["Aggregate rating"].copy()


In [22]:
# Clean/convert types for categorical
for c in feature_cols_cat:
    X[c] = X[c].astype(str)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [25]:
# Preprocessor
from sklearn.preprocessing import OneHotEncoder
numeric_tf = Pipeline(steps=[
    ("scaler", StandardScaler(with_mean=False))  # with_mean=False to play nice with sparse concat
])

categorical_tf = OneHotEncoder(handle_unknown="ignore", sparse_output=True)

pre = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, feature_cols_num),
        ("cat", categorical_tf, feature_cols_cat),
    ],
    remainder="drop",
    sparse_threshold=1.0
)


In [26]:
# --------- Define models ----------
models = {
    "LinearRegression": Pipeline(steps=[
        ("pre", pre),
        ("model", LinearRegression())
    ]),
    "DecisionTree": Pipeline(steps=[
        ("pre", pre),
        ("model", DecisionTreeRegressor(random_state=42, min_samples_leaf=5))
    ]),
    "RandomForest": Pipeline(steps=[
        ("pre", pre),
        ("model", RandomForestRegressor(
            n_estimators=300, random_state=42, n_jobs=-1, min_samples_leaf=3
        ))
    ]),
}

In [28]:
# --------- Train & Evaluate ----------
def eval_regression(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

metrics = []
best_name, best_r2 = None, -1e9
fitted_models = {}